Recreation of https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [41]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import save
from torch import load
from os import path

#import torch.utils.tensorboard as tb

from PIL import Image

from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import random 
import os, math

import gc

import pdb
from skimage import io 


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

Reproducibility

In [42]:
# Set manual seed.
def runRamdomSeed():
    torch.manual_seed(234)
    np.random.seed(234)
    random.seed(234)
    # Disabling the benchmarking feature with torch.backends.cudnn.benchmark = False 
    # causes cuDNN to deterministically select an algorithm, possibly at the cost of reduced performance.
    torch.backends.cudnn.benchmark = False 

runRamdomSeed()

First we need to import our data.

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Then we need to establish how we will transform the data to fit into NN

In [44]:
unused_data_transforms = transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [45]:

class OurDataset(Dataset):
    def __init__(self, data_transforms=None, valid=False):
        self.data_transforms = data_transforms
        if not valid:
          csv_path = "/content/drive/Shareddrives/GeoTracking_AI/image_data.csv"
          self.image_path = "/content/drive/Shareddrives/GeoTracking_AI/images"
        if valid:
          csv_path = "/content/drive/Shareddrives/GeoTracking_AI/image_valid_data.csv"
          self.image_path = "/content/drive/Shareddrives/GeoTracking_AI/valid_images"
        
        full_csv_frame = pd.read_csv(csv_path)
        csv_frame = full_csv_frame[["img_id", "city_id", "heading"]]
        
        self.pd_frame = csv_frame

        np_frame = csv_frame.to_numpy()

        

    def __len__(self):
        return self.pd_frame.iloc[:, 0].size


    def __getitem__(self, idx):
        image_path = self.image_path + "/" + str(self.pd_frame.iloc[idx, 0]).zfill(5) 
        img1 = Image.open(image_path + "_" + str( (self.pd_frame.iloc[idx, 2]) ).zfill(3) + ".jpg")
        img2 = Image.open(image_path + "_" + str( (self.pd_frame.iloc[idx, 2] + 120)%360 ).zfill(3) + ".jpg")
        img3 = Image.open(image_path + "_" + str( (self.pd_frame.iloc[idx, 2] + 240)%360 ).zfill(3) + ".jpg")
      
        img1 = torch.from_numpy(np.asarray(img1))     # convert to PyTorch Tensor
        img2 = torch.from_numpy(np.asarray(img2))  
        img3 = torch.from_numpy(np.asarray(img3)) 


        if self.data_transforms != None:
          img1 = self.data_transforms(img1)
          img2 = self.data_transforms(img2)
          img3 = self.data_transforms(img3)
        else:
          # Change from 640, 640, 3 to 3, 640, 640
          img1 = np.asarray(img1).transpose(-1, 0, 1)
          img2 = np.asarray(img2).transpose(-1, 0, 1)
          img3 = np.asarray(img3).transpose(-1, 0, 1)
        
        # img1 = torch.from_numpy(np.asarray(img1))     # convert to PyTorch Tensor (covered in transforms?)
        # img2 = torch.from_numpy(np.asarray(img2))     
        # img3 = torch.from_numpy(np.asarray(img3))     


        return (img1, img2, img3) , self.pd_frame.iloc[idx, 1]


Set Arguments

In [54]:
class Args(object):
    pass

args = Args();

args.learning_rate = .001
args.max_epochs = 10
args.batch_size = 64


Start by grabbing the already existing ResNet18 library

In [67]:
class Model(nn.Module):
  def __init__(self):
      super(Model, self).__init__()
      self.conv1 = nn.Conv2d(3, 6, 5)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
      self.conv2 = nn.Conv2d(6, 8, 5)
      self.conv3 = nn.Conv2d(8, 8, 5)
      self.conv4 = nn.Conv2d(8, 8, 5)
      self.conv5 = nn.Conv2d(8, 8, 5)
      self.conv6 = nn.Conv2d(8, 8, 5)
      self.conv7 = nn.Conv2d(8, 8, 5)
      self.conv8 = nn.Conv2d(8, 8, 5)
      self.fc1 = nn.Linear(3200, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)            # Note that output has dimension 10
  
  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = self.pool(F.relu(self.conv3(x)))
      x = self.pool(F.relu(self.conv4(x)))
      x = F.relu(self.conv5(x))
      x = F.relu(self.conv6(x))
      x = F.relu(self.conv7(x))
      x = F.relu(self.conv8(x))
      x = x.view(-1, 3200)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x 

model_ft = Model()

model_ft = model_ft.to(device)

In [68]:
def save_model(model, name):
  if (path.exists(name + ".pth")): raise Exception("already exists")
  else: save(model.state_dict(), name + ".pth")


In [69]:
train_dataset = OurDataset(data_transforms=None,valid=False)

trainloader = torch.utils.data.DataLoader(train_dataset, 
                                          batch_size = args.batch_size, 
                                          shuffle = True, 
                                          num_workers = 2)

valid_dataset = OurDataset(data_transforms=None,valid=True)
validloader = torch.utils.data.DataLoader(valid_dataset, 
                                          batch_size = args.batch_size, 
                                          shuffle = True, 
                                          num_workers = 2)

Train new model

In [62]:
def train_model(args, model):


    model = model.to(device) 

    #transform = transforms.Compose([transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
   
    #train_dataset = OurDataset(data_transforms=transform)
    #valid_dataset = OurDataset(data_transforms=transform)

    
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
    criterion = nn.CrossEntropyLoss()

    prev_val_acc = 0
    running_loss = 0.0
    best_val_acc = -1000
    for epoch in range(args.max_epochs):     # will get interupted by convergence test if validation acc drops
      for i, data in enumerate(trainloader, 0):
        # print("started i loop", i)
        for j in range(3):
          img = data[0][j]
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = img.to(device), data[1].to(device)
          inputs = inputs.float()         # Broke our RAM (?)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          del inputs, labels, loss        # added to reduce RAM issues
          gc.collect()

        # if i % 1000 == 999:    # print every 1000 mini-batches
        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

        # print("finished i loop", i)
        # print()

    save_model(model, "/content/drive/Shareddrives/GeoTracking_AI/custom_attempt1")

        
    print('Finished Training')

In [63]:
model_ft = train_model(args, model_ft)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

[1,    50] loss: 0.158


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

[2,    50] loss: 0.150


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

...

[8,    50] loss: 0.111


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

[9,    50] loss: 0.106


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

[10,    50] loss: 0.101
Finished Training


In [64]:
def accuracy_labels(preds, labels):
    return np.sum(preds == labels)/len(preds)

In [70]:
val_model = model_ft
val_dict = load("/content/drive/Shareddrives/GeoTracking_AI/custom_attempt1.pth")
val_model.load_state_dict(val_dict)

<All keys matched successfully>

In [71]:
  # Test on Val set
  test_batch_size = 64
  out_preds = []
  out_labels = []
  for (X,Y) in validloader:
    X = X[0].to(device)
    Y = Y.to(device)
    X = X.float()         # Broke our RAM (working at 64, not 256)
    y_pred = torch.argmax(val_model(X), dim = 1).tolist()
    print(y_pred)
    y_pred = map(int, y_pred)
    print(y_pred)
    out_preds.extend(list(y_pred))
    out_labels.extend(Y.tolist())
      
  this_val_acc = accuracy_labels(np.array(out_preds), np.array(out_labels))

  print('new_val_acc: %.3f'  %( this_val_acc))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorc

[4, 3, 6, 0, 3, 8, 8, 4, 5, 0, 1, 3, 1, 8, 7, 5, 1, 4, 8, 3, 1, 6, 8, 4, 7, 1, 1, 7, 5, 3, 0, 5, 4, 3, 5, 7, 3, 8, 3, 1, 7, 0, 4, 5, 3, 7, 1, 0, 6, 3, 0, 0, 7, 7, 8, 0, 1, 0, 5, 5, 4, 3, 8, 7]
[0, 0, 0, 5, 0, 0, 5, 0, 6, 0, 3, 5, 1, 5, 1, 5, 0, 7, 0, 1, 4, 8, 0, 4, 3, 8, 7, 0, 0, 5, 0, 3, 5, 4, 8, 1, 1, 6, 5, 7, 1, 4, 0, 3, 0, 8, 7, 8, 0, 0, 0, 7, 1, 5, 0, 7, 6, 3, 5, 1, 5, 7, 1, 3]
[4, 1, 1, 0, 3, 0, 5, 5, 0, 0, 0, 8, 1, 4, 3, 4, 6, 0, 1, 1, 7, 5, 0, 7, 8, 3, 0, 1, 0, 0, 1, 3, 1, 8, 1, 8, 3, 6, 7, 5, 7, 1, 5, 1, 5, 1, 7, 1, 7, 3, 7, 0, 4, 6, 8, 6, 1, 6, 5, 0, 5, 0, 3, 5]
[0, 0, 7, 8, 6, 4, 0, 7, 0, 0, 7, 0, 4, 0, 7, 0, 3, 7, 7, 1, 3, 1, 4, 0, 4, 3, 5, 7, 0, 1, 3, 7, 0, 7, 0, 3, 8, 3, 7, 4, 1, 0, 0, 3, 3, 4, 3, 8, 0, 4, 5, 4, 0, 0, 0, 1, 6, 6, 0, 7, 0, 3, 8, 0]
[5, 4, 8, 7, 7, 1, 1, 3, 7, 5, 1, 5, 0, 3, 1, 7, 0, 7, 6, 3, 0, 3, 1, 0, 0, 1, 3, 6, 3, 8, 6, 8, 0, 6, 8, 7, 8, 1, 3, 5, 7, 5, 0, 0, 1, 4, 7, 8, 1, 4, 4, 0, 4, 6, 3, 1, 5, 1, 0, 0, 0, 0, 0, 3]
[1, 6, 3, 0, 3, 5, 6, 0, 8, 3, 1, 0

Report:

* Made our own dataset - DONE

* Transfer Model - 4/15
  * Hyper-parameter testing
* Model from Scratch (do auto-tuning or hand-tune)
  * Hyper-parameter testing
* Have one of two models work well (compared to human benchmark)

Write Report - 4/23-4/30

* Visualization that we can see images, their correct labels, model's guess for label
* Percentage correct for each city bar graph